In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!mkdir .kaggle

In [3]:
import json
token = {"username":"cameronwebster","key":"5e70b65a76b1214dee43d83e7143f395"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
  json.dump(token, file)

In [4]:
!mkdir ~/.kaggle
!echo '{"username":"cameronwebster","key":"5e70b65a76b1214dee43d83e7143f395"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json && pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!rm -r cassava-leaf-disease-classification
!mkdir cassava-leaf-disease-classification
!kaggle competitions download -c cassava-leaf-disease-classification -p cassava-leaf-disease-classification

     |████████████████████████████████| 61kB 4.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=c2309af12ccbf1bcfc9136f5a85b14ab203be435013c61f5852ba51b8051def9
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10
rm: cannot remove 'cassava-leaf-disease-classification': No such file or directory
100% 5.75G/5.76G [01:06<00:00, 70.5MB/s]
100% 5.76G/5.76G [01:06<00:00, 92.8MB/s]


In [5]:
from zipfile import ZipFile
file_name = '/content/cassava-leaf-disease-classification/cassava-leaf-disease-classification.zip'

with ZipFile(file_name, 'r') as zip:

  print("Extracting new files")
  zip.extractall()
  print("done!")

Extracting new files
done!


In [6]:
import tensorflow as tf
import os
from functools import partial
!pip install -q wandb
import wandb
from matplotlib import pyplot as plt

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, InputLayer, 
    Conv2D, MaxPooling2D, ZeroPadding2D, LeakyReLU, Dropout, BatchNormalization
    )

from keras.layers.experimental.preprocessing import (
    RandomFlip, RandomRotation, RandomTranslation, RandomZoom, RandomContrast
    )

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
    )
!pip install -q tensorflow_addons
from tensorflow_addons.optimizers import MultiOptimizer
from tensorflow.keras.optimizers.schedules import ExponentialDecay

     |████████████████████████████████| 2.0MB 8.1MB/s 
     |████████████████████████████████| 133kB 38.8MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 163kB 38.7MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 706kB 7.8MB/s 


## Parameters, Logging Initiation

In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
SHUFFLE_SEED = 42
BATCH_SIZE = 16 
# IMAGE_SIZE = [336, 336]
CROP_FACTOR = 0.7
IMAGE_SIZE = [int((520*CROP_FACTOR)-4)] * 2
EPOCHS = 100
LR = 0.001
CLASSES = 5
TRAIN_SPLIT = 0.8

# Learning Rate
BASE_LR_INIT = 0.00001
HEAD_LR_INIT = 0.001
RAMPUP = 10
EXP_DECAY = 0.1

# Early Stopping
MIN_DELTA_ES = 0.001
PATIENCE_ES = 15

# Reduce LR on Plateau
FACTOR_RLR = 0.8
PATIENCE_RLR = 3
MIN_DELTA_RLR = 0.0001
MIN_LR = 0.0000001

RECORD_PATH = tf.io.gfile.glob("train_tfrecords/*.tfrec")
# SAVE_PATH = '/content/drive/Shareddrives/2040_Midterm_Project/Classy_Classifiers/model_checkpoints/inception_models/'
SAVE_PATH = '/content/drive/MyDrive/DATA_2040/'
SAVE_NAME = "March20-v3-Inception-{epoch:02d}-{val_loss:.2f}.hdf5"

wandb.init(project="InceptionNet")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)

    image = tf.image.central_crop(image, CROP_FACTOR)

    image = image / 255.0 # recale by 255 for inputs between 0 and 1

    return image

In [9]:
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "target": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        label = tf.one_hot(label, 5)
        return image, label
    return image

In [10]:
def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [11]:
data_augmentation = Sequential([
    # RandomCrop(height=100, width=100),
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(15),
    RandomTranslation(height_factor=0.2, width_factor=0.2),
    # RandomZoom(height_factor=(-0.2,0.2), width_factor=(-0.2,0.2)),
    RandomContrast(factor=0.2)
])

## Shuffling, Splitting, Batching, Augmenting train set, Caching val set, Prefetching

splitting from: https://stackoverflow.com/questions/60704335/how-to-create-train-test-validation-split-of-tf-data-dataset-in-tf-2-1-0

shuffling, batchingm caching, prefetching from https://www.kaggle.com/tt195361/splitting-tensorflow-dataset-for-validation

In [12]:
def get_datasets(filenames, labeled=True, train_split=TRAIN_SPLIT):

    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048, seed=SHUFFLE_SEED)

    # First, split our train dataset into train and validation

    # Size of dataset
    n = sum(1 for _ in dataset)
    n_train = int(n * train_split)
    n_valid = n - n_train

    train_dataset = dataset.take(n_train)
    val_dataset = dataset.skip(n_train).take(n_valid)

    train_dataset = train_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.batch(BATCH_SIZE)

    # Augment the training set
    train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y), 
                  num_parallel_calls=AUTOTUNE)
    
    # From Docs: "The first time the dataset is iterated over, its elements 
    # will be cached either in the specified file or in memory. Subsequent 
    # iterations will use the cached data."
    val_dataset = val_dataset.cache()
    
    # prefetch "allows later elements to be prepared while the current 
    # element is being processed"
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

    return train_dataset, val_dataset

In [13]:
train_dataset, val_dataset = get_datasets(RECORD_PATH)

## (Optional) Inspect Records

In [14]:
# image_batch, label_batch = next(iter(dataset_original))

# def show_batch(image_batch, label_batch):
#     plt.figure(figsize=(10, 10))
#     for n in range(25):
#         ax = plt.subplot(5, 5, n + 1)
#         plt.imshow(image_batch[n] / 255.0)
#         plt.title(f"{label_batch[n]}")
#         plt.axis("off")

# show_batch(image_batch.numpy(), label_batch.numpy())

# for raw_record in dataset_original.take(1):
    # example = tf.train.Example()
    # example.ParseFromString(raw_record)
    # print(example)

## Sequential Model

In [15]:
# Re-loads the MobileNet model without the top or FC layers
# tf.keras.backend.clear_session()

inception_base = InceptionV3(include_top=False, 
                            weights='imagenet', 
                            input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3),
                            classes=CLASSES,
                            classifier_activation=None)
    
model = tf.keras.Sequential([
    InputLayer(input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3)),              

    inception_base,
    GlobalAveragePooling2D(),

    # Dense(1024, kernel_regularizer='l2'),
    # BatchNormalization(),
    # LeakyReLU(),
    # Dropout(0.5),

    Dropout(0.5),
    Dense(512, kernel_regularizer='l2', activation='relu'),
    BatchNormalization(),
    
    # Dropout(0.5),
    # Dense(128, kernel_regularizer='l2', activation='relu'),
    # BatchNormalization(),
    
    # Dropout(0.5),
    # Dense(32, kernel_regularizer='l2', activation='relu'),
    # BatchNormalization(),

    Dense(CLASSES,activation='softmax')
])

inception_base.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 360, 360, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 179, 179, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 179, 179, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 179, 179, 32) 0           batch_normalization[0][0]        
______________________

In [16]:
checkpoint = ModelCheckpoint(SAVE_PATH+SAVE_NAME,
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             save_weights_only = False,
                             verbose = 1)

# class CheckpointNoOpt(tf.keras.callbacks.Callback):

#     def on_epoch_end(self, epoch, logs=None):
#         model.save(filepath=SAVE_PATH + NAME_BASE + f"-{epoch}-.hdf5", 
#                    overwrite=False, 
#                    include_optimizer=False
# )
# checkpoint = CheckpointNoOpt()

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = MIN_DELTA_ES, 
                          patience = PATIENCE_ES,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = FACTOR_RLR,
                              patience = PATIENCE_RLR,
                              verbose = 1,
                              min_delta = MIN_DELTA_RLR,
                              min_lr = MIN_LR)

# def lr(epoch, start_lr, rampup_epochs, exp_decay):
#   if epoch < rampup_epochs:
#     return start_lr
#   else:
#     return start_lr * math.exp(-exp_decay * epoch)

# def schedule_base(epoch):
#   return lr(epoch, BASE_LR_INIT, RAMPUP, EXP_DECAY)

# def schedule_head(epoch):
#   return lr(epoch, HEAD_LR_INIT, RAMPUP, EXP_DECAY)

# scheduler_base = LearningRateScheduler(schedule_base, verbose=0)
# scheduler_head = LearningRateScheduler(schedule_head, verbose=0)

# scheduler_base = ExponentialDecay(initial_learning_rate=BASE_LR_INIT, 
#                                   decay_steps=20, decay_rate=.1)
# scheduler_head = ExponentialDecay(initial_learning_rate=HEAD_LR_INIT, 
#                                   decay_steps=20, decay_rate=.1)

# var_list_base = model.layers[0]
# var_list_head = model.layers[1:]
# opt_base = Adam(learning_rate=scheduler_base)
# opt_head = Adam(learning_rate=scheduler_head)

# optimizers_and_layers = [(opt_base, var_list_base),(opt_head, var_list_head)]
# opt = MultiOptimizer(optimizers_and_layers)


model.compile(loss = 'categorical_crossentropy',
              # optimizer = opt,
              optimizer = Adam(learning_rate=LR),
              metrics = ['accuracy'])

callbacks = [checkpoint,earlystop, reduce_lr, wandb.keras.WandbCallback()]
# callbacks = [checkpoint, earlystop, wandb.keras.WandbCallback()]

model.build(input_shape=(BATCH_SIZE,IMAGE_SIZE[0],IMAGE_SIZE[1],3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 9, 9, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
batch_normalization_94 (Batc (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 22,856,485
Trainable params: 22,821,029
Non-trainable params: 35,456
_____________________________________

In [17]:
history = model.fit(
    train_dataset,
    # class_weight = class_weights,
    epochs = EPOCHS,
    validation_data = val_dataset,
    callbacks = callbacks,
    verbose = 1#,
    #batch_size=BATCH_SIZE#,
    #steps_per_epoch = STEPS_PER_EPOCH,
    #validation_steps = VALIDATION_STEPS
)

Epoch 1/100
1070/1070 [==============================] - 349s 300ms/step - loss: 4.5196 - accuracy: 0.5716 - val_loss: 1.8509 - val_accuracy: 0.3481

Epoch 00001: val_loss improved from inf to 1.85092, saving model to /content/drive/MyDrive/DATA_2040/March20-v3-Inception-01-1.85.hdf5
Epoch 2/100
1070/1070 [==============================] - 286s 262ms/step - loss: 1.2679 - accuracy: 0.6103 - val_loss: 1.0339 - val_accuracy: 0.6584

Epoch 00002: val_loss improved from 1.85092 to 1.03390, saving model to /content/drive/MyDrive/DATA_2040/March20-v3-Inception-02-1.03.hdf5
Epoch 3/100
1070/1070 [==============================] - 282s 258ms/step - loss: 1.0882 - accuracy: 0.6414 - val_loss: 1.0210 - val_accuracy: 0.6645

Epoch 00003: val_loss improved from 1.03390 to 1.02102, saving model to /content/drive/MyDrive/DATA_2040/March20-v3-Inception-03-1.02.hdf5
Epoch 4/100
1070/1070 [==============================] - 283s 259ms/step - loss: 0.9799 - accuracy: 0.6674 - val_loss: 1.2497 - val_accur

KeyboardInterrupt: ignored

In [19]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: ignored

In [18]:
wandb.finish()

epoch,81
loss,0.34634
accuracy,0.88532
val_loss,0.36333
val_accuracy,0.87734
lr,5e-05
_runtime,23246
_timestamp,1616359788
_step,81
best_val_loss,0.36333
best_epoch,81


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
val_loss,█▄▄▃▃▃▂▂▂▂▂▃▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▅▆▆▆▇▇▇▇▇▅▇█▇▇▇██▇████████████████████
lr,███▇▇▇▇▇▇▇▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [ ]:
model.evaluate(val_dataset)